# AtomMan Atoms Class Demonstration

Class Atoms represents a dictionary of atomic properties.  All data is stored in a single numpy array to minimize memory cost, and properties can be accessed either with controlled methods that account for data types and units (Section 1), or directly through the use of numpy views (Section 2).

The underlying code can be found in atomman/Atoms.py.

__Library Imports__

In [1]:
#Standard library imports
from collections import OrderedDict

#External library imports
import numpy as np

#atomman imports
from atomman import Atoms

## 1. Atoms basics

This section describes the basics for interacting with atomic properties stored in an Atoms instance.  The methods described in this section allow for safe controlled access and assignment of the properties.  These methods should handle most standard uses.  

### 1.1 Initilization and simple functionality

Initilizing an Atoms instance has the following optional arguments:

- __natoms__ = number of atoms in the list.  The number of atoms is fixed after creation.  Default value is 1.

- __prop__ = dictionary containing per-atom properties.

- __prop_dtype__ = a dictionary for explicitly defining the data types for the different properties.  Optional when initializing with prop (Section 1.3), but mandatory when initilizing with data and view (Section 2.3). 

- __nvals__ = number of numeric property values given to each atom.  This automatically increases if needed.  As an argument, it can offer slight performance improvements if you know how many property values you will assign.  Default value is 30.

- __data__ = a numpy data array to be assigned at the core of the Atoms instance.  For advanced use only (Section 2.3).

- __view__ = a dictionary of views pointing to the data array. For advanced use only (Section 2.3).

In [2]:
#initialize an Atoms instance with 10 atoms
#atoms = Atoms(natoms=5) is equivalent
atoms = Atoms(5)

The default values of all properties are set to zero. By default, each atom is pre-assigned two properties:

- __atype__ = integer atom type

- __pos__ = (3x1) float array coordinate position  

Converting to a string displays the atype and pos for all atoms in a formatted manner.

In [3]:
print atoms

     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       0 |   0.000 |   0.000 |   0.000
      1 |       0 |   0.000 |   0.000 |   0.000
      2 |       0 |   0.000 |   0.000 |   0.000
      3 |       0 |   0.000 |   0.000 |   0.000
      4 |       0 |   0.000 |   0.000 |   0.000


The number of atoms and the number of atom types can be retrieved with natoms and natypes, respectively.

In [4]:
print "atoms.natoms ->", atoms.natoms
print "atoms.natypes ->", atoms.natypes

atoms.natoms -> 5
atoms.natypes -> 0


Property values can also be assigned during initilization using a dictionary and the prop argument.

In [5]:
prop_dict = {'atype': range(10,0,-1), 'pos': np.ones((10,3))}

#atoms = Atoms(natoms=10, prop=prop_dict) is equivalent
atoms = Atoms(10, prop_dict)

print atoms
print "atoms.natoms ->", atoms.natoms
print "atoms.natypes ->", atoms.natypes

     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |      10 |   1.000 |   1.000 |   1.000
      1 |       9 |   1.000 |   1.000 |   1.000
      2 |       8 |   1.000 |   1.000 |   1.000
      3 |       7 |   1.000 |   1.000 |   1.000
      4 |       6 |   1.000 |   1.000 |   1.000
      5 |       5 |   1.000 |   1.000 |   1.000
      6 |       4 |   1.000 |   1.000 |   1.000
      7 |       3 |   1.000 |   1.000 |   1.000
      8 |       2 |   1.000 |   1.000 |   1.000
      9 |       1 |   1.000 |   1.000 |   1.000
atoms.natoms -> 10
atoms.natypes -> 10


### 1.2 The prop() method

Atom property values can be set and retrieved using the prop() method.  The keyword arguments are:

- __a_id__ = atom index.

- __key__ = atom property name.

- __value__ = value(s) to assign to properties associated with a_id and/or term.

- __dtype__ = data type to explicitly set or retrieve value as. 

If no arguments given, returns a list of the assigned property keys. Otherwise, a_id and/or key must be specified. The key specifies which property, and the a_id which atom(s) to access. With no value argument, prop() returns which value(s) are associated with the given a_id and/or key. With a value argument, the value is saved according to the given a_id and/or key.

Calling prop with no arguments returns a list of assigned property keys.

In [6]:
#Testing prop with no arguments
print 'atoms.prop() ->', atoms.prop()

atoms.prop() -> ['atype', 'pos']


Specific property values can be retrieved by calling prop() with key and/or a_id. All values returned with prop() are "safe", i.e. they are copies of the Atoms' data values.

In [7]:
#Retrieving a property array for all atoms
print "atoms.prop(key='atype') ->", atoms.prop(key='atype')
print "atoms.prop(key='pos') ->" 
print atoms.prop(key='pos')
print

#Retrieving a single atom
print 'atom_1 = atoms.prop(a_id=1)'
atom_1 = atoms.prop(a_id=1)
print "atom_1.prop(key='atype') ->", atom_1.prop(key='atype')
print "atom_1.prop(key='pos') ->", atom_1.prop(key='pos')
print

#Retrieving a property of a single atom.
print "atoms.prop(a_id=2, key='pos') ->", atoms.prop(a_id=2, key='pos')

atoms.prop(key='atype') -> [10  9  8  7  6  5  4  3  2  1]
atoms.prop(key='pos') ->
[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]

atom_1 = atoms.prop(a_id=1)
atom_1.prop(key='atype') -> 9
atom_1.prop(key='pos') -> [ 1.  1.  1.]

atoms.prop(a_id=2, key='pos') -> [ 1.  1.  1.]


Property values can be set using the prop method either for all atoms at once, or on a per-atom basis.  This is done by supplying value after key, and/or a_id.

In [8]:
#Setting all atypes at once (uses numpy broadcasting)
print "atoms.prop(key='atype', value=[1]) sets all atypes to be one"
atoms.prop(key='atype', value=[1])
print

#Setting all pos at once
print "atoms.prop(key='pos', value=5*np.random.rand(10,3)) sets all pos to be random between 0-5"
atoms.prop(key='pos', value=5*np.random.rand(10,3))
print

#Setting atypes individually 
print "atoms.prop(a_id=0, key='atype', value=2) makes atype for atom at index 0 to be 2"
atoms.prop(a_id=0, key='atype', value=2)
print 

#Setting one atom to be equal to another
print "atoms.prop(a_id=7, value=atoms.prop(a_id=1)) copies atom 1 index info to atom 7"
atoms.prop(a_id=7, value=atoms.prop(a_id=1))
print

#Display atoms after changes
print atoms

atoms.prop(key='atype', value=[1]) sets all atypes to be one

atoms.prop(key='pos', value=5*np.random.rand(10,3)) sets all pos to be random between 0-5

atoms.prop(a_id=0, key='atype', value=2) makes atype for atom at index 0 to be 2

atoms.prop(a_id=7, value=atoms.prop(a_id=1)) copies atom 1 index info to atom 7

     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       2 |   3.142 |   2.563 |   1.385
      1 |       1 |   1.751 |   1.835 |   0.430
      2 |       1 |   3.353 |   4.320 |   1.376
      3 |       1 |   1.422 |   4.509 |   0.548
      4 |       1 |   3.571 |   0.796 |   2.498
      5 |       1 |   3.568 |   0.028 |   3.472
      6 |       1 |   1.781 |   3.950 |   2.223
      7 |       1 |   1.751 |   1.835 |   0.430
      8 |       1 |   0.051 |   3.733 |   0.974
      9 |       1 |   3.716 |   4.812 |   4.900


Additional atomic properties (beyond atype and pos) can be freely defined using the prop() method.  The value for each new property can be of any regular shape and of a data type that can be converted into a float (bool, int, and long work but complex, unicode, and str do not.)  The shape and data type are set to match the original value and are identical for all atoms.  

In [9]:
#Trying to retrieve any property that has not been assigned returns None
print "atoms.prop(key='Not-assigned') ->", atoms.prop(key='Not-assigned')
print

#Properties are assigned by giving a key and value
print "atoms.prop(key='scalar-int', value=np.arange(10)) assigns a scalar integer value"
atoms.prop(key='scalar-int', value=np.arange(10))
print "atoms.prop(key='scalar-int') ->", atoms.prop(key='scalar-int')
print

#If assigned to one atom, all atoms gain that property with default zero values
print "atoms.prop(a_id=5, key='scalar-bool', value=True) assigns True to atom at index 5 (zero=False in Bool)"
atoms.prop(a_id=5, key='scalar-bool', value=True)
print "atoms.prop(key='scalar-bool') ->",atoms.prop(key='scalar-bool')
print

#Shapes of higher order data structures are retained
print "atoms.prop(key='vector-float', value=np.random.rand(10,2)) creates a 2D vector value"
atoms.prop(key='vector-float', value=np.random.rand(10,2))
print "atoms.prop(a_id=1, key='vector-float') ->", atoms.prop(a_id=1, key='vector-float') 
print 

print "atoms.prop(key='matrix-int', value=[np.eye(4) for i in xrange(10)]) gives all atoms a 4x4 identity matrix"
atoms.prop(key='matrix-int', value=[np.eye(4) for i in xrange(10)])
print "atoms.prop(a_id=2, key='matrix-int') ->"
print atoms.prop(a_id=2, key='matrix-int') 
print 

print "atoms.prop() lists all assigned property keys"
print "atoms.prop() ->", atoms.prop()


atoms.prop(key='Not-assigned') -> None

atoms.prop(key='scalar-int', value=np.arange(10)) assigns a scalar integer value
atoms.prop(key='scalar-int') -> [0 1 2 3 4 5 6 7 8 9]

atoms.prop(a_id=5, key='scalar-bool', value=True) assigns True to atom at index 5 (zero=False in Bool)
atoms.prop(key='scalar-bool') -> [False False False False False  True False False False False]

atoms.prop(key='vector-float', value=np.random.rand(10,2)) creates a 2D vector value
atoms.prop(a_id=1, key='vector-float') -> [ 0.07281863  0.67086797]

atoms.prop(key='matrix-int', value=[np.eye(4) for i in xrange(10)]) gives all atoms a 4x4 identity matrix
atoms.prop(a_id=2, key='matrix-int') ->
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]

atoms.prop() lists all assigned property keys
atoms.prop() -> ['atype', 'pos', 'scalar-int', 'scalar-bool', 'vector-float', 'matrix-int']


### 1.3 dtype conversion

The data type for each property is set when the property is first defined using the prop() method.  This is either done implicitly using the property value's data type, or explicitly using the dtype argument.  The dtype argument and prop() method also allows for property values to be returned converted to a specified data type.  Note that this output conversion does not change the property's stored dtype.  

In [10]:
#Create small system for demo purposes
atoms = Atoms(natoms=4)

#Create list of ints
print 'test = [30, 50, 10, 56]'
test = [30, 50, 10, 56]
print

#Implicit data type assignment
print "atoms.prop(key='implicit', value=test)"
atoms.prop(key='implicit', value=test)
print "atoms.prop(key='implicit') ->", atoms.prop(key='implicit')
print

#Explicit data type assignment
print "atoms.prop(key='explicit', value=test, dtype='float')"
atoms.prop(key='explicit', value=test, dtype='float')
print "atoms.prop(key='explicit') ->", atoms.prop(key='explicit')
print

#Showing data value retrieval in a specified data type
print "atoms.prop(a_id=1, key='implicit', dtype=bool) ->", atoms.prop(a_id=1, key='implicit', dtype=bool)
print "atoms.prop(key='implicit', dtype=float) ->", atoms.prop(key='implicit', dtype=float)
print "atoms.prop(key='implicit') ->", atoms.prop(key='implicit')

test = [30, 50, 10, 56]

atoms.prop(key='implicit', value=test)
atoms.prop(key='implicit') -> [30 50 10 56]

atoms.prop(key='explicit', value=test, dtype='float')
atoms.prop(key='explicit') -> [ 30.  50.  10.  56.]

atoms.prop(a_id=1, key='implicit', dtype=bool) -> True
atoms.prop(key='implicit', dtype=float) -> [ 30.  50.  10.  56.]
atoms.prop(key='implicit') -> [30 50 10 56]


The assigned data type offers control when reading in values. Once set for a property, the data type cannot be changed with prop().  Also, any values being assigned are checked if they are compatible with the assigned data type.

In [11]:
print 'test2 = [0,1,2,3]'
test2 = [0,1,2,3]
print "atoms.prop(key='test2', value=test2, dtype=int)"
atoms.prop(key='test2', value=test2, dtype=int)
print "atoms.prop(key='test2') ->", atoms.prop(key='test2')
print 

#If property already exists, the dtype argument must be None or match the property's dtype
print "Testing assignment of dtype to test2 using prop()"
dtypes = [bool, int, float, None]
for dtype in dtypes:
    print "  atoms.prop(key='test2', value=test2, dtype=%s)" % dtype
    try:
        atoms.prop(key='test2', value=test2, dtype=dtype)
        print "    atoms.prop(key='test2') ->", atoms.prop(key='test2')
    except AssertionError as detail:
        print '    AssertionError raised:', detail
print

#Values being assigned must be compatible with the property's dtype    
print "Testing assignment of dtype to test2 using prop()"
values = [False, 2, 2.3, 'a']
for value in values:
    print "  atoms.prop(a_id=3, key='test2', value=%s)" % value
    try:
        atoms.prop(a_id=3, key='test2', value=value)
        print "    atoms.prop(key='test2') ->", atoms.prop(key='test2')
    except AssertionError as detail:
        print '    AssertionError raised:', detail
    except ValueError as detail:
         print '    ValueError raised:', detail        

test2 = [0,1,2,3]
atoms.prop(key='test2', value=test2, dtype=int)
atoms.prop(key='test2') -> [0 1 2 3]

Testing assignment of dtype to test2 using prop()
  atoms.prop(key='test2', value=test2, dtype=<type 'bool'>)
    AssertionError raised: dtype already assigned as <type 'int'>
  atoms.prop(key='test2', value=test2, dtype=<type 'int'>)
    atoms.prop(key='test2') -> [0 1 2 3]
  atoms.prop(key='test2', value=test2, dtype=<type 'float'>)
    AssertionError raised: dtype already assigned as <type 'int'>
  atoms.prop(key='test2', value=test2, dtype=None)
    atoms.prop(key='test2') -> [0 1 2 3]

Testing assignment of dtype to test2 using prop()
  atoms.prop(a_id=3, key='test2', value=False)
    atoms.prop(key='test2') -> [0 1 2 0]
  atoms.prop(a_id=3, key='test2', value=2)
    atoms.prop(key='test2') -> [0 1 2 2]
  atoms.prop(a_id=3, key='test2', value=2.3)
    AssertionError raised: value is incompatible with int data type
  atoms.prop(a_id=3, key='test2', value=a)
    AssertionError rai

Data types can also be explicitly assigned during initilization using the prop and prop_dtype arguments.  Specifying None for a property's dtype will use the implicit dtype for that property.  Note that the dtypes of atype and pos are pre-defined to int and float respectively.

In [12]:
#initilizing with prop and prop_unit
print "prop =      {'atype':[1,1,1,1], 'pos':[[1.,1.,1.], [2.,2.,2.], [3.,3.,3.], [4.,4.,4.]], 'u_test':[1,1,1,1]}"
print "prop_dtype ={'atype': None,     'pos': None,                                            'u_test': float}"
prop = {'atype': [1, 1, 1, 1], 
        'pos': [[1., 1., 1.],
                [2., 2., 2.],
                [3., 3., 3.],
                [4., 4., 4.]],
       'u_test': [1, 1, 1, 1]}

prop_dtype = {'atype': None, 'pos': None, 'u_test': float}
print
atoms = Atoms(natoms=4, prop=prop, prop_dtype=prop_dtype)
print atoms

#checking unit control for Ecoh
print "u_test's dtype explicitly set to be float:"
print "atoms.prop(key='u_test') ->", atoms.prop(key='u_test')

prop =      {'atype':[1,1,1,1], 'pos':[[1.,1.,1.], [2.,2.,2.], [3.,3.,3.], [4.,4.,4.]], 'u_test':[1,1,1,1]}
prop_dtype ={'atype': None,     'pos': None,                                            'u_test': float}

     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   1.000 |   1.000 |   1.000
      1 |       1 |   2.000 |   2.000 |   2.000
      2 |       1 |   3.000 |   3.000 |   3.000
      3 |       1 |   4.000 |   4.000 |   4.000
u_test's dtype explicitly set to be float:
atoms.prop(key='u_test') -> [ 1.  1.  1.  1.]


## 2. Atoms advanced usage

For advanced users, Atoms also allows for direct access to the underlying data without going through the prop method.  This is achieved primarily through the use of numpy views.  The advantage of this is that allows for users to specify new routines and algorithms capable of fully taking advantage of numpy's vectorized functions.  The disadvantage is the loss of safety and control from the prop() method.  Thus, proper usage is in the hands of the user.

### 2.1 data, view, and dtype

The core of an Atoms instance is:

- __data__ = a numpy ndarray of floats of size (natoms x nvals).

- __view__ = an OrderedDict consisting of numpy arrays that are views of data.

- __dtype__ = an OrderedDict containing each property's assigned dtype.

When properties are first assigned to an atom, the values are flattened to a one dimensional array, converted to floats, and stored in data.  A view is created of the property consisting of the correct size and shape, but values as floats. The appropriate data type is stored in the dtype dictionary.

In [13]:
#Create new Atoms instance
print "atoms = Atoms(natoms=5, nvals=10)"
atoms = Atoms(natoms=5, nvals=10)
print

#Show underlying data array
print "All data stored in atoms.data as floats"
print "atoms.data ->"
print atoms.data
print

#Show the atype view
print "atoms.view accesses a view of the underlying data"
print "atoms.view['atype'] ->"
print atoms.view['atype']
print

#Show the atype dtype
print "data type information kept in atoms.dtype"
print "atoms.dtype['atype'] ->", atoms.dtype['atype']

atoms = Atoms(natoms=5, nvals=10)

All data stored in atoms.data as floats
atoms.data ->
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]

atoms.view accesses a view of the underlying data
atoms.view['atype'] ->
[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]

data type information kept in atoms.dtype
atoms.dtype['atype'] -> int32


Numpy views are new arrays that point back to the original data.  Because of this, changing the values in a view will change the values in data.

In [14]:
#Setting atype values using it's view
print "Changing atype values using a view:"
print "atype = atoms.view['atype']"
print "for i in xrange(len(atype)):"
print "    atype[i] = 1"

atype = atoms.view['atype']
for i in xrange(len(atype)):
    atype[i] = 1

print
print "atoms.data ->"
print atoms.data

Changing atype values using a view:
atype = atoms.view['atype']
for i in xrange(len(atype)):
    atype[i] = 1

atoms.data ->
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


__POINT OF CAUTION #1:__ Make certain that you assign values to the elements of a view, not the name of the view.

In [15]:
#POINT OF CAUTION #1 demonstration
print "pos = atoms.view['pos']"
pos = atoms.view['pos']
print

#Correct assignment
print "Correct assignment:"
print "Assigning values to elements of a view will change the values"
print "pos[:] = np.ones((5,3))" 
pos[:] = np.ones((5,3))
print "pos ->"
print pos
print "atoms.data ->"
print atoms.data
print

#incorrect assignment
print "Incorrect assignment:" 
print "Assigning values to the name of a view will change what the name points to"
print "pos = np.zeros((5,3))" 
pos = np.zeros((5,3))
print "pos ->"
print pos
print "atoms.data ->"
print atoms.data

pos = atoms.view['pos']

Correct assignment:
Assigning values to elements of a view will change the values
pos[:] = np.ones((5,3))
pos ->
[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]
atoms.data ->
[[ 1.  1.  1.  1.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  0.  0.  0.  0.  0.  0.]]

Incorrect assignment:
Assigning values to the name of a view will change what the name points to
pos = np.zeros((5,3))
pos ->
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
atoms.data ->
[[ 1.  1.  1.  1.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  0.  0.  0.  0.  0.  0.]]


The default stored data type can be changed using the dtype dictionary.  While this allows for incorrectly assigned data types to be changed, it is more appropriate to explicitly assign a troublesome data type when first created using the prop() method.

In [16]:
#Assignment is prevented if value cannot be converted to the stored dtype
print "atoms.prop(key='int-test', value=[1,2,3,4,5], dtype=int)"
atoms.prop(key='int-test', value=[1,2,3,4,5], dtype=int)

print "Trying to assign a float to int-test with atoms.prop(a_id=2, key='scalar-int', value=3.2):"
print "atoms.prop(a_id=2, key='int-test', value=3.2) ->", 
try:
    atoms.prop(a_id=2, key='int-test', value=3.2)
    print atoms.prop(key='int-test')
except AssertionError as detail:
    print 'AssertionError raised:', detail
print

#The stored dtype can be changed using dtype
print "Changing int-test's dtype to float and trying again:"
print "atoms.dtype['int-test'] = float"
atoms.dtype['int-test'] = float
print "atoms.prop(a_id=2, key='int-test', value=3.2) ->", 
try:
    atoms.prop(a_id=2, key='int-test', value=6.2)
    print atoms.prop(key='int-test')
except AssertionError as detail:
    print 'AssertionError raised:', detail    

atoms.prop(key='int-test', value=[1,2,3,4,5], dtype=int)
Trying to assign a float to int-test with atoms.prop(a_id=2, key='scalar-int', value=3.2):
atoms.prop(a_id=2, key='int-test', value=3.2) -> AssertionError raised: value is incompatible with int data type

Changing int-test's dtype to float and trying again:
atoms.dtype['int-test'] = float
atoms.prop(a_id=2, key='int-test', value=3.2) -> [ 1.   2.   6.2  4.   5. ]


__POINT OF CAUTION #2:__ Since all data is stored as floats and only converted with the prop() method, direct access of data, view and dtype offers no data type control.

In [17]:
print "Changing int-test's dtype back to int does not affect the underlying data:"
print "atoms.dtype['int-test'] = int"
atoms.dtype['int-test'] = int
print "atoms.prop('int-test') ->", atoms.prop(key='int-test')
print "atoms.data ->"
print atoms.data
print 

print "Likewise, value assignment with data or view not data type controlled:"
print "atoms.view['atype'][0] = 14.234"
atoms.view['atype'][0] = 14.234
print "atoms.prop(key='atype') ->", atoms.prop(key='atype')
print "atoms.data ->"
print atoms.data

Changing int-test's dtype back to int does not affect the underlying data:
atoms.dtype['int-test'] = int
atoms.prop('int-test') -> [1 2 6 4 5]
atoms.data ->
[[ 1.   1.   1.   1.   1.   0.   0.   0.   0.   0. ]
 [ 1.   1.   1.   1.   2.   0.   0.   0.   0.   0. ]
 [ 1.   1.   1.   1.   6.2  0.   0.   0.   0.   0. ]
 [ 1.   1.   1.   1.   4.   0.   0.   0.   0.   0. ]
 [ 1.   1.   1.   1.   5.   0.   0.   0.   0.   0. ]]

Likewise, value assignment with data or view not data type controlled:
atoms.view['atype'][0] = 14.234
atoms.prop(key='atype') -> [14  1  1  1  1]
atoms.data ->
[[ 14.234   1.      1.      1.      1.      0.      0.      0.      0.      0.   ]
 [  1.      1.      1.      1.      2.      0.      0.      0.      0.      0.   ]
 [  1.      1.      1.      1.      6.2     0.      0.      0.      0.      0.   ]
 [  1.      1.      1.      1.      4.      0.      0.      0.      0.      0.   ]
 [  1.      1.      1.      1.      5.      0.      0.      0.      0.      0.   ]]

### 2.2 Index access

Specific atoms within an Atoms instance can be accessed using index reference, i.e. [].  This returns a new Atoms instance where data is a view to the indexed atoms of the original Atoms instance.  This offers an alternate way of accessing and changing values on a per atom basis.

In [18]:
#Index access and assignment of atoms
atoms = Atoms(natoms = 10, prop={'atype': range(10), 'pos': np.random.rand(10,3)})
print "atoms ->"
print atoms

#Access with specified indexes
print "atoms[2:7] ->"
print atoms[2:7]

#Assign one atom to another
print "atoms[5] = atoms[9]"
atoms[5] = atoms[9]
print "atoms ->"
print atoms

#Iterate over atoms for retrieval
print "for atom in atoms:"
print "    print atom.prop(key='atype'), ->"
for atom in atoms:
     print atom.prop(key='atype'),
print 
print 

#Iterate over atoms for assignment using prop
print "for atom in atoms:"
print "    atom.prop(key='atype', value=1)"
for atom in atoms:
     atom.prop(key='atype', value=1)
print "atoms.prop(key='atype') ->", atoms.prop(key='atype')
print 

#Iterate over atoms for assignment using view
print "for atom in atoms:"
print "    atom.view['atype'][:] = 2"
for atom in atoms:
     atom.view['atype'][:] = 2
print "atoms.prop(key='atype') ->", atoms.prop(key='atype')

atoms ->
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       0 |   0.700 |   0.321 |   0.783
      1 |       1 |   0.882 |   0.391 |   0.088
      2 |       2 |   0.024 |   0.701 |   0.356
      3 |       3 |   0.556 |   0.541 |   0.891
      4 |       4 |   0.525 |   0.807 |   0.808
      5 |       5 |   0.547 |   0.056 |   0.135
      6 |       6 |   0.825 |   0.575 |   0.311
      7 |       7 |   0.192 |   0.774 |   0.961
      8 |       8 |   0.292 |   0.291 |   0.263
      9 |       9 |   0.354 |   0.884 |   0.756
atoms[2:7] ->
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       2 |   0.024 |   0.701 |   0.356
      1 |       3 |   0.556 |   0.541 |   0.891
      2 |       4 |   0.525 |   0.807 |   0.808
      3 |       5 |   0.547 |   0.056 |   0.135
      4 |       6 |   0.825 |   0.575 |   0.311
atoms[5] = atoms[9]
atoms ->
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       0 |   0.700 |   0.321 |   0.783
      1 |       1 |   0.882 |   0.39

__POINT OF CAUTION #3:__ New properties should only be assigned to the original Atoms instance and any other instances created using index access regenerated.  Otherwise, multiple properties may be assigned to the same data values, or the different instances may no longer point to the same data.

In [19]:
atoms = Atoms(natoms = 5, nvals = 5, prop={'atype':[1], 'pos':np.random.rand(5,3)})
print "atoms ->"
print atoms

print "atoms_2 = atoms[2:4]"
atoms_2 = atoms[2:4]
print "atoms_2 ->"
print atoms_2

#test assignment to atoms_2
print "If a new property is assigned to atoms_2, the values will be in data, but atoms instance will not know of it"
print "atoms_2.prop(key='new', value=[2,2])"
atoms_2.prop(key='new', value=[2,2])
print "atoms.prop() ->", atoms.prop()
print "atoms_2.prop() ->", atoms_2.prop()
print "atoms.data->"
print atoms.data

atoms ->
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.495 |   0.664 |   0.661
      1 |       1 |   0.533 |   0.688 |   0.780
      2 |       1 |   0.490 |   0.657 |   0.187
      3 |       1 |   0.835 |   0.255 |   0.419
      4 |       1 |   0.295 |   0.505 |   0.600
atoms_2 = atoms[2:4]
atoms_2 ->
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.490 |   0.657 |   0.187
      1 |       1 |   0.835 |   0.255 |   0.419
If a new property is assigned to atoms_2, the values will be in data, but atoms instance will not know of it
atoms_2.prop(key='new', value=[2,2])
atoms.prop() -> ['atype', 'pos']
atoms_2.prop() -> ['atype', 'pos', 'new']
atoms.data->
[[ 1.          0.49466348  0.66390285  0.66065643  0.        ]
 [ 1.          0.53323706  0.68757009  0.78037151  0.        ]
 [ 1.          0.49035246  0.65736573  0.1871213   2.        ]
 [ 1.          0.83488062  0.25534398  0.41933581  2.        ]
 [ 1.          0.29459904  0.50499565  0.

### 2.3 Initilization using data, view, and prop_dtype

An Atoms instance can also be initialized by directly supplying a data array, and view and dtype dictionaries.  This allows for exact control over defining the core of an Atoms instance.  Extra care has to be taken as there is no check that the views and data array point to the same data.  

In [20]:
atoms = Atoms(natoms = 5, nvals = 5, prop={'atype':[1], 'pos':np.random.rand(5,3)})
print "atoms ->"
print atoms

#this code is equivalent to newatoms = atoms[index]
print "Create newatoms by assigning data, view, and prop_dtype associated with atom index 3:"
index = 3
view = OrderedDict()
for k, v in atoms.view.iteritems():
    view[k] = v[index]
newatoms = Atoms(data=atoms.data[index], view=view, prop_dtype=atoms.dtype)
print "newatoms ->"
print newatoms

print "atoms[3] ->"
print atoms[index]

atoms ->
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.498 |   0.331 |   0.395
      1 |       1 |   0.714 |   0.670 |   0.729
      2 |       1 |   0.663 |   0.485 |   0.939
      3 |       1 |   0.183 |   0.731 |   0.908
      4 |       1 |   0.301 |   0.722 |   0.323
Create newatoms by assigning data, view, and prop_dtype associated with atom index 3:
newatoms ->
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.183 |   0.731 |   0.908
atoms[3] ->
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.183 |   0.731 |   0.908
